In [1]:
import pandas as pd
import numpy as np
import random
from quantnote import BacktestEngine, Strategy

In [2]:
db_path = '/Users/j.nahmgoong/Documents/git/DB/MainDB.db'

engine = BacktestEngine(db_path)

Loading DB...
DB loaded in 44.53 seconds
[initialize] is executed in 51.08 seconds


In [3]:
engine.show_sample_strategy()

class myStrategy(Strategy):
    def __init__(self):
        super().__init__()

    def compute_target(self, universe_list):
        target_weight = { }
        for ticker in universe_list:
            target_weight[ticker] = 1
        target_weight = self.normalize(target_weight)
        return target_weight

    def custom_factor(self, ticker, ftype):
        if ftype == 'marketcap':
            return self.get_value('metric',ticker,'marketcap')
        else:
            assert False



In [4]:
non_cap_norm = ['tbvps',
'sps',
'ros',
'roe',
'roa',
'ps1',
'ps',
'price',
'prefdivis',
'pe1',
'pe',
'pb',
'payoutratio',
'fcfps',
'epsdil',
'eps',
'dps',
'divyield',
'de',
'currentratio',
'bvps',
'marketcap']

In [5]:
class myStrategy(Strategy):
    def __init__(self, config):
        super().__init__()
        self.config = config
        
        print(self.config)

    def compute_target(self, universe_list):
        target_weight = { }
        
        input1 = self.config['input1']
        input2 = self.config['input2']
        unary = self.config['unary']
        binary = self.config['binary']
        unary_out = self.config['unary_out']
        
        factor_series_1 = self.compute_factor_series(universe_list, input1, type=unary)
        factor_series_2 = self.compute_factor_series(universe_list, input2, type=unary)
        
        if binary == 'add':
            factor_series = factor_series_1 + factor_series_2
        elif binary == 'sub':
            factor_series = factor_series_1 - factor_series_2
        elif binary == 'mul':
            factor_series = factor_series_1 * factor_series_2
        elif binary == 'div':
            factor_series = factor_series_1 / factor_series_2
        
        factor_series = factor_series.dropna().sort_values(ascending=False)
        assert len(factor_series) > 0.5*len(universe_list)
        
        if unary_out == 'raw':
            pass
        elif unary_out == 'rank':
            factor_series = factor_series.rank(ascending=False)
        elif unary_out == 'zscore':
            factor_series = (factor_series-factor_series.mean())/factor_series.std()
        
        factor_series = factor_series.dropna().sort_values(ascending=False)
        top = int(len(factor_series.index)/5)
        
        for ticker in factor_series.index:
            weight = factor_series[ticker]
            if weight > 0:
                target_weight[ticker] = weight
            
        target_weight = self.normalize(target_weight)
        return target_weight

    def custom_factor(self, ticker, ftype):
        if ftype == 'marketcap':
            return self.get_value('metric',ticker,'marketcap')
        else:
            if ftype in non_cap_norm:
                x = self.get_value('fundamentals',ticker,ftype)
            else:
                x = self.get_value('fundamentals',ticker,ftype)/self.get_value('fundamentals',ticker,'marketcap')
            return x



In [6]:
input_list = list(engine.cache['fundamentals']['AAPL'].columns[5:])
unary_list = ['raw', 'rank', 'zscore']
binary_list = ['add', 'sub', 'mul', 'div']

result_df = pd.DataFrame({
    'sharpe':[],
    'cagr':[],
    'vol':[],
    'input1':[],
    'input2':[],
    'unary':[],
    'binary':[],
    'unary_out':[],
})

len(input_list)

105

In [ ]:
idx = 1
while True:
    x1, x2 = random.sample(input_list,2)
    unary = random.sample(unary_list,1)[0]
    binary = random.sample(binary_list,1)[0]
    unary_out = random.sample(unary_list,1)[0]
    
    config = {
    'input1':x1,
    'input2':x2,
    'unary':unary,
    'binary':binary,
    'unary_out':unary_out}
    
    print('\n')
    print('Computing {}th factor'.format(idx))
    mys = myStrategy(config)
    try:
        engine.run_backtest(mys, '2016-01-01','2021-08-31',period='Q')
        engine.stat(bench=None)
        print(engine.stat_dic)
        
        sharpe = engine.stat_dic['sharpe']
        cagr = engine.stat_dic['cagr']
        vol = engine.stat_dic['vol']
        
        result_df = result_df.append({
            'sharpe':sharpe,
            'cagr':cagr,
            'vol':vol,
            'input1':x1,
            'input2':x2,
            'unary':unary,
            'binary':binary,
            'unary_out':unary_out,
        },ignore_index=True)
        
        result_df.to_parquet('result.parquet')

        idx += 1
    except:
        print('Passed!')
        
    result_df = result_df.sort_values('sharpe', ascending=False)
    print('Current best sharpe')
    try:
        print(result_df.iloc[:3])
    except:
        pass




Computing 1th factor
{'input1': 'pe', 'input2': 'evebit', 'unary': 'raw', 'binary': 'div', 'unary_out': 'raw'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00


C:\Users\j.nahmgoong\Anaconda3\lib\site-packages\quantnote\strategy.py:65: RuntimeWarning: invalid value encountered in double_scalars
  target_weight = {ticker:target_weight[ticker]/target_sum for ticker in target_weight}


Passed!
Current best sharpe
Empty DataFrame
Columns: [sharpe, cagr, vol, input1, input2, unary, binary, unary_out]
Index: []


Computing 1th factor
{'input1': 'intexp', 'input2': 'bvps', 'unary': 'zscore', 'binary': 'add', 'unary_out': 'raw'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:1.426 / time elapsed:58.1 ===
[run_backtest] is executed in 58.39 seconds
{'cagr': 0.05974459003901591, 'vol': 0.24018902435752007, 'sharpe': 0.24873988392611315}
Current best sharpe
    sharpe      cagr       vol  input1 input2   unary binary unary_out
0  0.24874  0.059745  0.240189  intexp   bvps  zscore    add       raw


Computing 2th factor
{'input1': 'assetsavg', 'input2': 'ps', 'unary': 'zscore', 'binary': 'sub', 'unary_out': 'rank'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:2.439 / time elapsed:134.3 ===
[run_backtest] is executed in 135.06 seconds
{'cagr': 0.15739063675311904, 

Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:2.731 / time elapsed:92.9 ===
[run_backtest] is executed in 93.59 seconds
{'cagr': 0.1777078803700367, 'vol': 0.20578375132617238, 'sharpe': 0.8635661427338123}
Current best sharpe
      sharpe      cagr       vol     input1       input2   unary binary  \
12  0.863566  0.177708  0.205784       sgna  payoutratio    rank    add   
0   0.856389  0.158251  0.184789    ebitusd    tangibles    rank    add   
1   0.848389  0.157391  0.185517  assetsavg           ps  zscore    sub   

   unary_out  
12    zscore  
0        raw  
1       rank  


Computing 14th factor
{'input1': 'cashneq', 'input2': 'sharesbas', 'unary': 'rank', 'binary': 'add', 'unary_out': 'rank'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:2.052 / time elapsed:135.7 ===
[run_backtest] is executed in 136.55 seconds
{'cagr': 0.12709001323221028, 'vol': 0.1936390855348

=== date:2021-08-31 00:00:00 / total_asset:2.526 / time elapsed:120.6 ===
[run_backtest] is executed in 121.28 seconds
{'cagr': 0.1636168400087881, 'vol': 0.18243687120943838, 'sharpe': 0.8968408574654583}
Current best sharpe
      sharpe      cagr       vol   input1        input2   unary binary  \
22  0.896841  0.163617  0.182437      ps1  investmentsc  zscore    mul   
0   0.863566  0.177708  0.205784     sgna   payoutratio    rank    add   
1   0.856389  0.158251  0.184789  ebitusd     tangibles    rank    add   

   unary_out  
22      rank  
0     zscore  
1        raw  


Computing 24th factor
{'input1': 'accoci', 'input2': 'revenueusd', 'unary': 'rank', 'binary': 'add', 'unary_out': 'rank'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:2.059 / time elapsed:134.8 ===
[run_backtest] is executed in 135.56 seconds
{'cagr': 0.12756631160608561, 'vol': 0.18916493604043785, 'sharpe': 0.6743655260656537}
Current best sharpe
     s

...

C:\Users\j.nahmgoong\Anaconda3\lib\site-packages\quantnote\strategy.py:65: RuntimeWarning: invalid value encountered in double_scalars
  target_weight = {ticker:target_weight[ticker]/target_sum for ticker in target_weight}


Passed!
Current best sharpe
     sharpe      cagr       vol         input1     input2 unary binary  \
0  1.069502  0.228492  0.213644             pb      fxusd   raw    add   
1  0.983807  0.200171  0.203466         debtnc  netincnci  rank    mul   
2  0.979247  0.184406  0.188314  investmentsnc     equity  rank    add   

  unary_out  
0       raw  
1    zscore  
2    zscore  


Computing 67th factor
{'input1': 'assetturnover', 'input2': 'netinccmn', 'unary': 'rank', 'binary': 'div', 'unary_out': 'zscore'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:1.461 / time elapsed:67.9 ===
[run_backtest] is executed in 68.29 seconds
{'cagr': 0.06562547984729385, 'vol': 0.19865489259096988, 'sharpe': 0.33034917484975623}
Current best sharpe
     sharpe      cagr       vol         input1     input2 unary binary  \
0  1.069502  0.228492  0.213644             pb      fxusd   raw    add   
1  0.983807  0.200171  0.203466         debtnc  netin

=== date:2021-08-31 00:00:00 / total_asset:1.540 / time elapsed:122.1 ===
[run_backtest] is executed in 122.79 seconds
{'cagr': 0.07365571017932994, 'vol': 0.25371076565783157, 'sharpe': 0.2903136963398161}
Current best sharpe
     sharpe      cagr       vol         input1     input2 unary binary  \
0  1.069502  0.228492  0.213644             pb      fxusd   raw    add   
1  0.983807  0.200171  0.203466         debtnc  netincnci  rank    mul   
2  0.979247  0.184406  0.188314  investmentsnc     equity  rank    add   

  unary_out  
0       raw  
1    zscore  
2    zscore  


Computing 78th factor
{'input1': 'ps1', 'input2': 'ncf', 'unary': 'raw', 'binary': 'mul', 'unary_out': 'zscore'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:1.653 / time elapsed:74.4 ===
[run_backtest] is executed in 74.96 seconds
{'cagr': 0.08460762465609226, 'vol': 0.22004899157275096, 'sharpe': 0.3844944893924675}
Current best sharpe
     sharpe      cag

Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:1.452 / time elapsed:57.6 ===
[run_backtest] is executed in 57.90 seconds
{'cagr': 0.0633409600451099, 'vol': 0.23084996162378918, 'sharpe': 0.2743815056306364}
Current best sharpe
     sharpe      cagr       vol         input1     input2 unary binary  \
0  1.069502  0.228492  0.213644             pb      fxusd   raw    add   
1  0.983807  0.200171  0.203466         debtnc  netincnci  rank    mul   
2  0.979247  0.184406  0.188314  investmentsnc     equity  rank    add   

  unary_out  
0       raw  
1    zscore  
2    zscore  


Computing 89th factor
{'input1': 'netmargin', 'input2': 'shareswa', 'unary': 'rank', 'binary': 'mul', 'unary_out': 'raw'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:2.339 / time elapsed:136.7 ===
[run_backtest] is executed in 137.46 seconds
{'cagr': 0.1499479274074256, 'vol': 0.1863960880601993, 'sha

Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:2.495 / time elapsed:122.5 ===
[run_backtest] is executed in 123.15 seconds
{'cagr': 0.16160886691412585, 'vol': 0.1815339268771919, 'sharpe': 0.8902405720747429}
Current best sharpe
     sharpe      cagr       vol         input1     input2 unary binary  \
0  1.069502  0.228492  0.213644             pb      fxusd   raw    add   
1  0.983807  0.200171  0.203466         debtnc  netincnci  rank    mul   
2  0.979247  0.184406  0.188314  investmentsnc     equity  rank    add   

  unary_out  
0       raw  
1    zscore  
2    zscore  


Computing 100th factor
{'input1': 'rnd', 'input2': 'cor', 'unary': 'rank', 'binary': 'add', 'unary_out': 'zscore'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:2.096 / time elapsed:92.7 ===
[run_backtest] is executed in 93.22 seconds
{'cagr': 0.13043800895557628, 'vol': 0.19535700649319174, 'sharpe':

In [ ]:
result_df

In [ ]:
1+1